In [ ]:
!pip install deepface

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Setup Repository
import os
os.chdir('/content')
CODE_DIR = 'encoder4editing'

!git clone https://github.com/eladrich/pixel2style2pixel.git $CODE_DIR
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
os.chdir(f'./{CODE_DIR}')

from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from utils.common import tensor2im
from models.psp import pSp  # we use the pSp framework to load the e4e encoder.

%load_ext autoreload
%autoreload 2

In [ ]:
#@title Select which experiment you wish to perform inference on: { run: "auto" }
experiment_type = 'unmasked_masked' #@param ['unmasked_masked','ffhq_encode', 'ffhq_frontalize', 'celebs_sketch_to_face', 'celebs_seg_to_face', 'celebs_super_resolution', 'toonify']

In [ ]:
#@title Download
def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory ../pretrained_models. """
    current_directory = os.getcwd()
    save_path = os.path.join(os.path.dirname(current_directory), CODE_DIR, "pretrained_models")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url

MODEL_PATHS = {
    "unmasked_masked" : {"id": "17BKYibSc43zLFZxeyAfsHMBkBE6TG7ml", "name": "unmasked_masked.pt"}
}

path = MODEL_PATHS[experiment_type]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"])
!{download_command}

In [ ]:
EXPERIMENT_DATA_ARGS = {
    "unmasked_masked": {
        "model_path": "pretrained_models/unmasked_masked.pt",
        "image_path": "notebooks/images/input_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    }
}

EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS[experiment_type]
img_transforms = EXPERIMENT_ARGS['transform']
resize_dims = (256, 256)

In [ ]:
model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
# update the training options
opts['checkpoint_path'] = model_path
if 'learn_in_w' not in opts:
    opts['learn_in_w'] = False
if 'output_size' not in opts:
    opts['output_size'] = 1024

In [ ]:
opts = Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

Loading pSp from checkpoint: pretrained_models/unmasked_masked.pt
Model successfully loaded!


In [ ]:
def run_on_batch(inputs, net, latent_mask=None):
    if latent_mask is None:
        result_batch = net(inputs.to("cuda").float(), randomize_noise=False)
    else:
        result_batch = []
        for image_idx, input_image in enumerate(inputs):
            # get latent vector to inject into our input image
            vec_to_inject = np.random.randn(1, 512).astype('float32')
            _, latent_to_inject = net(torch.from_numpy(vec_to_inject).to("cuda"),
                                      input_code=True,
                                      return_latents=True)
            # get output image with injected style vector
            res = net(input_image.unsqueeze(0).to("cuda").float(),
                      latent_mask=latent_mask,
                      inject_latent=latent_to_inject)
            result_batch.append(res)
        result_batch = torch.cat(result_batch, dim=0)
    return result_batch

In [ ]:
!unzip /content/drive/Shareddrives/AIProject/Dataset/celeba_unmasked_folder.zip -d /content/sample_data
!unzip /content/drive/Shareddrives/AIProject/Dataset/celeba_masked_folder.zip -d /content/sample_data

In [ ]:
import shutil
# shutil.rmtree("/content/sample_data/result_face")
!mkdir /content/sample_data/result_face

In [ ]:
def calc_distance(image_name,df):
  distance = -1
  for i in range(len(df['VGG-Face_cosine'])):
    if(image_name == df['identity'][i].split("/")[4] ):
      distance = df['VGG-Face_cosine'][i]    

  if(distance == -1):
    distance = 1

  return distance  

In [ ]:
'''
#replace file to mute warning
from deepface import DeepFace
import gdown
import os
import shutil
url = "https://drive.google.com/u/0/uc?id=1aS5SJCK9MLMyYPXmhAkwtn3cJhQGCbGj&export=download"
gdown.download(url, "/content/sample_data/DeepFace.py")
os.remove("/usr/local/lib/python3.7/dist-packages/deepface/DeepFace.py")
shutil.move("/content/sample_data/DeepFace.py","/usr/local/lib/python3.7/dist-packages/deepface/")
'''

In [ ]:
from torchvision.utils import save_image
masked_dir = "/content/sample_data/celeba_masked_folder/"
db_path = "/content/sample_data/celeba_unmasked_folder"
from deepface import DeepFace
count = 0
total_accuracy = 0
for masked_image_dir in os.listdir(masked_dir):
  if not masked_image_dir.endswith(".ipynb_checkpoints"):
    count += 1
    image = Image.open(masked_dir + masked_image_dir)
    image = image.convert("RGB")
    image = image.resize(resize_dims)
    image = img_transforms(image)
    with torch.no_grad():
      tic = time.time()
      images = run_on_batch(image.unsqueeze(0), net,None)
      result_image = images[0]
      result_image = (result_image+1)/2
      result_dir = "/content/sample_data/result_face/"+ masked_image_dir
      save_image(result_image,result_dir)
      toc = time.time()
      df = DeepFace.find(img_path=result_dir,db_path=db_path,enforce_detection=False)
      accuracy = 1 - calc_distance(masked_image_dir.split(".")[0],df)
      total_accuracy += accuracy
      print("masked_image :",masked_image_dir,"recognize_image:",df['identity'][0].split("/")[4],"Accuracy: ",accuracy)
    if(count >= 1000):
      break  

print("Accuracy of Model over {} images is : {}".format(count,total_accuracy/count))

In [ ]:
#ffhq psp 0.4820933278986273

In [ ]:
shutil.rmtree("/content/sample_data/celeba_unmasked_folder")
shutil.rmtree("/content/sample_data/celeba_masked_folder")

In [ ]:
%cd /content/sample_data
!apt-get install rar
!rar a "pkl.rar" "/content/sample_data/ffhq_unmasked_folder"